In [ ]:
"""(net1.py) open queueing network"""

from SimPy.Simulation import *
import random
import numpy
import math

## Useful extras ----------
def tablelookup(P):
    """Sample from i=0..n-1 with probabilities P[i]"""
    u = random.random()
    sumP = 0.0
    for i in range(len(P)):
        sumP += P[i]
        if u < sumP:
            return i

def conf(L):
    """confidence interval"""
    lower = numpy.mean(L) - 1.96*numpy.std(L)/math.sqrt(len(L))
    upper = numpy.mean(L) + 1.96*numpy.std(L)/math.sqrt(len(L))
    return (lower,upper)

## Model ----------
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = random.expovariate(lamb)
            yield hold, self, t

class Arrival(Process):
    """an arrival"""
    n = 0  # class variable (number in system)

    def run(self):
        Arrival.n += 1   # number in system
        arrivetime = now()
        G.numbermon.observe(Arrival.n)

	current = G.startnode
	while (current <> G.outnode):
            yield request,self,G.server[current]
            t = random.expovariate(G.mu[current])
            yield hold,self,t
            yield release,self,G.server[current]
            current = tablelookup(G.transition[current])

        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        delay = now()-arrivetime
        G.delaymon.observe(delay)

class G:
    server = 'dummy'
    delaymon = 'Monitor'
    numbermon = 'Monitor'
    startnode = 0
    outnode = 3
    mu = [1/1.0, 1/2.0, 1/1.0]
    transition = [[0.0, 0.5, 0.5, 0.0],
                  [0.0, 0.0, 0.8, 0.2],
                  [0.2, 0.0, 0.0, 0.8]]

def model(N, lamb, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = [Resource(1),Resource(2),Resource(3)]
    G.delaymon = Monitor()
    G.numbermon = Monitor()
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    return(W,L)

## Experiment ----------------

allW = []
allL = []
allLambdaEffective = []
for k in range(50):
    seed = 123*k
    result = model(N=10000, lamb=0.5,
                  maxtime=2000000, rvseed=seed)
    allW.append(result[0])
    allL.append(result[1])
    allLambdaEffective.append(result[1]/result[0])

print "Estimate of W:",numpy.mean(allW)
print "Conf int of W:",conf(allW)
print "Estimate of L:",numpy.mean(allL)
print "Conf int of L:",conf(allL)
print "Estimate of LambdaEffective:",numpy.mean(allLambdaEffective)
print "Conf int of LambdaEffective:",conf(allLambdaEffective)